In [ ]:
# dependencies import

#common
import os
import re
import time
import pathlib
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
from matplotlib.patches import Rectangle
from matplotlib.axes import Axes

# ML
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Dropout,
    concatenate, Flatten, Dense, UpSampling2D,
    BatchNormalization
)

# my
from create_logger import *
import custom_modules as dw
import models

logger = logging.getLogger(f'main.ae_train')

# Настраиваемые параметры

In [ ]:
PATH_TO_SAVE_MODEL = pathlib.Path(f'test/')
MODEL_VERSION = 1

In [ ]:
XSHIFT = 200 # сдвиг по оси х для визуализации

# настройка параметров выборок
dataset_desc = {'train': [dw.DataPart(run_name='run_1', height=60),
                          dw.DataPart(run_name='run_2', height=60)],
                'val': [dw.DataPart(run_name='run_1', xy=(0,60), height=20),
                        dw.DataPart(run_name='run_2', xy=(0,60), height=20)],
                'test': [dw.DataPart(run_name='run_1', xy=(0,80)),
                        dw.DataPart(run_name='run_2', xy=(0,80))]}

# Вспомогательные функции

In [ ]:
def get_dataset_as_squeezed_numpy_decorator(func):
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        return np.squeeze(np.array(list(gen)))
    return wrapper
    
@get_dataset_as_squeezed_numpy_decorator
def get_dataset(df: pd.DataFrame, descs: list[dw.DataPart]):
    """
    Подготовить датасет на основании списка описаний dw.DataPart.
    Готовит датасет для одной из 3 выборок за раз - тренировочной, тестовой или валидационной
    """
    df = df.copy()
    df = dw.cast_df_to_2d(df)
    for i, ((_, time), (_, amp)) in enumerate(zip(df['Time'].items(), df['Amplitude'].items())):
        df['Time_x_Amplitude', i] = time * amp
    df = df.drop(['Time', 'Amplitude', 'DefectDepth'], axis=1)
    arr = dw.standardize_data(df.to_numpy())
    df = pd.DataFrame(data=arr, index=df.index, columns=df.columns)
    df = dw.cast_df_to_3d(df)
    
    generators_list = []
    for desc in descs:
        temp_df = df.copy()
        if not desc.run_name is None:
            temp_df = temp_df.loc[desc.run_name]
        temp_df = dw.crop_df(temp_df, desc.xy, desc.width, desc.height)
        generators_list.append(dw.get_crop_generator(dw.df_to_numpy(temp_df), 
                                                     desc.crop_size, 
                                                     desc.crop_step, 
                                                     desc.augmentations))
    return itertools.chain(*generators_list)    



# def generator_to_squeezed_numpy(gen):
#     return  np.squeeze(np.array(list(gen)))

def draw_plot(data: list[dict], title = 'Результат одного замера УЗ-датчика', x_label = 'Время', y_label = 'Амплитуда', fontsize = 25, path_to_save = None):
    """
    Нарисовать график

    Параметры
    ----------
    data: list[dict]
        Список словарей. Каждый словарь хранит
        данные и все мараметры для рисования конкретного графика.
        Все графики будут нарисованы на 1 полотне. 'data' параметр обязателен в 
        каждом словаре. Он хранит список из 1 или 2 массивов - это x и y для 

    Пример: 
    time = [1,2,3,4,5]
    amp = [4,-4,5,-5,6]
    draw_plot([{'data':[time, amp], 'marker':'o', 'lw':3, 'label':'Исходные данные', 'ms':10, 'mfc':'black'}])
    Все ключевые слова взяты из функции plt.plot()

    plt.plot()
    Если указать в словаре только 'data' параметр без остальных, то оформление будет сделано автоматом

    """
    fig, ax = plt.subplots()
    fig.set_figwidth(18)
    fig.set_figheight(10)
    
    fig.set_facecolor('#37474f')
    ax.set_facecolor('black')


    for item in data:
        if not 'data' in item:
            raise ValueError('Каждый словарь должен содержать ключ "data"')
        if list(item.keys()) == ['data']:
            ax.plot(*item['data'], marker='o', lw=3, label='Исходные данные', ms=10, mfc='black')
        else:
            ax.plot(*item['data'], **item)
    

    fig.suptitle(title, fontsize=fontsize+5, c='#cacaca')
    ax.legend(fontsize = fontsize, labelcolor='#cacaca', facecolor='black')
    ax.set_xlabel(x_label, fontsize=fontsize, c='#cacaca')
    ax.set_ylabel(y_label, fontsize=fontsize, c='#cacaca')
    
    ax.tick_params(axis='both', labelsize = fontsize)
    ax.grid(True, which='major', axis='both', lw=1.5)
    ax.grid(True, which='minor', axis='both', ls='--')
    
    ax.minorticks_on()
    
    ax.tick_params(axis = 'both', which = 'major', length = 8, width = 4, colors='#cacaca')
    ax.tick_params(axis = 'both', which = 'minor', length = 4, width = 2, labelleft=True, colors='#cacaca', labelsize=fontsize-8)
    
    #ax.xaxis.set_minor_locator(MultipleLocator(0.05))
    #ax.yaxis.set_minor_locator(MultipleLocator(0.05))
    #ax.xaxis.set_minor_formatter(FormatStrFormatter("%.3f"))
    #ax.yaxis.set_minor_formatter(FormatStrFormatter("%.3f"))
    
    ax.set_facecolor
    #plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.1, hspace=0.1)
    if not path_to_save is None:
        plt.savefig(path_to_save, bbox_inches='tight')
    plt.tight_layout()
    plt.show()
    plt.close()

# Чтение и подготовка данных

## Оригинальные данные

In [ ]:
df = dw.get_data_df('data/original_data')
display(df)

In [ ]:
display(dw.cast_df_to_2d(df))

## Данные после перемножения времен на амплитуды

In [ ]:
test_df = dw.cast_df_to_2d(df).copy()
for i, ((_, time_val), (_, amp_val)) in enumerate(zip(test_df['Time'].items(), test_df['Amplitude'].items())):
    test_df['Time_x_Amplitude', i] = time_val * amp_val
display(test_df)

In [ ]:
# нарисовать графики до обработки
for i in range(10):
    draw_plot([{'data':[test_df.loc['run_1',0,i]['Time'], test_df.loc['run_1',0,i]['Amplitude']]},
               
              ])

In [ ]:
# нарисовать графики после перемножения времен и амплитуд
for i in range(10):
    draw_plot([{'data':[test_df.loc['run_1',0,i]['Time_x_Amplitude']]},
              
              ])

## Визуализация частей выборок

In [ ]:
# show parts took for learning
all_rects = []
rects_colors = {'train':'red', 'val':'green', 'test':'yellow', 'all':'orange'}

temp_df = df.copy()
temp_df = temp_df.map(lambda x: x[-1])
temp_df = dw.roll_df(temp_df, XSHIFT, 1)

for dataset_name, descs in dataset_desc.items():
    for desc in descs:
        xy = desc.xy
        width = desc.width
        height = desc.height
        run_name = desc.run_name
    
        if desc.run_name is None:
            if width is None:
                width = temp_df.shape[1]
            if height is None:
                height = temp_df.shape[0]
        else:
            if height is None:
                height = temp_df.loc[desc.run_name].shape[0] - xy[1]
            if width is None:
                width = temp_df.loc[desc.run_name].shape[1]
                
            for i, item in enumerate(df.index):
                if item[0] == desc.run_name:
                    xy = (xy[0], i+xy[1])
                    break
                    
        all_rects.append(Rectangle(xy, width, height, facecolor=rects_colors[dataset_name], alpha=0.5))

if all_rects:
    dw.draw_defects_map_with_rectangles_owerlap(temp_df, all_rects, title = f'The parts took for learning from {set(temp_df.index.get_level_values("File"))} {rects_colors}')

del temp_df

# Распределение данных по выборкам

In [ ]:
train = get_dataset(df, dataset_desc['train'])
val= get_dataset(df, dataset_desc['val'])
test = get_dataset(df, dataset_desc['test'])

In [ ]:
logger.debug('Dataset parts shapes')
logger.debug(f'{train.shape=}')
logger.debug(f'{val.shape=}')
logger.debug(f'{test.shape=}')

# Создание и обучение модели автокодировщика

In [ ]:
def get_AE(learning_rate):
    solver = keras.optimizers.Adam(learning_rate) # оптимизатор
    loss_funcs = keras.losses.MeanSquaredError() # функция потерь/ошибки
    metrics = [keras.metrics.MeanSquaredError(name='MeanSquaredError')] # отслеживаемые метрики

    enc_input = layers.Input((32,), name='enc_input')
    d_1 = layers.Dense(32, activation='tanh')(enc_input)
    
    d_2 = layers.Dense(16, activation='tanh')(d_1)
    d_3 = layers.Dense(16, activation='tanh')(d_1)
    
    hidden_state_output = layers.Dense(8, activation='tanh', name='hidden_state_output')(concatenate([d_2, d_3], axis=1))
    
    d_5 = layers.Dense(16, activation='tanh')(hidden_state_output)
    d_6 = layers.Dense(16, activation='tanh')(hidden_state_output)
    
    dec_output = layers.Dense(32, activation='tanh', name='dec_output')(concatenate([d_5, d_6], axis=1))
    
    model = keras.Model(enc_input, dec_output, name='AE')
    model.compile(optimizer=solver, loss=loss_funcs, metrics=metrics)
    return model

## Вывести параметры модели и её граф

In [ ]:
# activations = 'sigmoid' #'relu' 'x' 'tanh'
# losses = 
model = get_AE(0.001)
print(model.summary())
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)

## Обучение модели

## Не настраиваемые параметры

In [ ]:
# размер входных и выходных данных
ENCODED_SIZE = min([layer.output.shape[1] for layer in model.layers]) 
DECODED_SIZE = model.layers[-1].output.shape[1]
PATH_TO_SAVE_MODEL = PATH_TO_SAVE_MODEL/f'AE/encoded_to_{ENCODED_SIZE}'
PATH_TO_SAVE_MODEL_PROGRESS = PATH_TO_SAVE_MODEL/'logs'
MODEL_NUMBER = 1
MIN_TRAIN_LOSS = 1
MIN_VAL_LOSS = 1

if not os.path.exists(PATH_TO_SAVE_MODEL):
    os.makedirs(PATH_TO_SAVE_MODEL)

if not os.path.exists(PATH_TO_SAVE_MODEL_PROGRESS):
    os.makedirs(PATH_TO_SAVE_MODEL_PROGRESS)

# все имеющиеся модели с такими же ENCODED_SIZE и DECODED_SIZE
all_ae_models = [path.name for path in PATH_TO_SAVE_MODEL.parent.rglob('*.keras') 
                 if re.search(fr'in\({DECODED_SIZE}\)_hid\({ENCODED_SIZE}\)', path.name)]

# если уже есть такая же архитектура модели сделать MODEL_VERSION такой же
# а MODEL_NUMBER на 1 больше чем имеющаяся
if all_ae_models:
    min_train_loss = min([float(re.findall(fr'train=(.+),val', name)[0]) for name in all_ae_models])
    min_val_loss = min([float(re.findall(fr',val=(.+),test', name)[0]) for name in all_ae_models])
    min_model_number = max([int(re.findall(fr'id=v{MODEL_VERSION:04}n(\d+)_', name)[0]) for name in all_ae_models])

    if MODEL_NUMBER <= min_model_number:
        MODEL_NUMBER = min_model_number+1
        
    if MIN_TRAIN_LOSS >= min_train_loss:
        MIN_TRAIN_LOSS = min_train_loss

    if MIN_VAL_LOSS >= min_val_loss:
        MIN_VAL_LOSS = min_val_loss



print(f'{ENCODED_SIZE=}')
print(f'{DECODED_SIZE=}')
print(f'{PATH_TO_SAVE_MODEL=}')
print(f'{PATH_TO_SAVE_MODEL_PROGRESS=}')
print(f'{MODEL_VERSION=}')
print(f'{MODEL_NUMBER=}')
print(f'{MIN_TRAIN_LOSS=}')
print(f'{MIN_VAL_LOSS=}')

### Настройка и создание коллбэков

In [ ]:
callback_params = {
    # остановка обучения если модель перестала учиться
    'EarlyStopping': {
        'monitor': 'val_loss', # отслеживаемый параметр 
        'min_delta': 0.00001, # минимальное улучшение параметра за cur_patience
        'patience': 6, # кол-во эпох без улучшений
        'restore_best_weights': False,  # сохранять ли веса нейронки с лучшими результатами
    },

    # уменьшение шага сходимости, если модель стала мендленно учиться
    'ReduceLROnPlateau': {
        'monitor' : 'loss', # отслеживаемый параметр 
        'factor' : 0.2, # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
        'patience' : 3, # кол-во эпох без улучшений
        'verbose' : 0, # выводить ли прогресс изменения шага сходимости в его процессее
        'min_delta' : 0.0001, # порог изменения отслеживаемого значения
        'cooldown' : 1, # количество эпох до возобновления работы после изменения шага сходимости
        'min_lr' : 0# минимальное значение шага сходимости
    },
}

# Создание и настройка колбэков
callback_list = [] # массив колбэков до подачи в колбек "callbacklist"
# остановка обучения если модель перестала учиться
callback_list.append(keras.callbacks.EarlyStopping(**callback_params['EarlyStopping']))
# уменьшение шага сходимости, если модель стала мендленно учиться
callback_list.append(keras.callbacks.ReduceLROnPlateau(**callback_params['ReduceLROnPlateau']))

### Обучение моделей

In [ ]:
def save_learning_params(seed, lrate, batch_size, learn_time, history, callback_params, dataset_desc, train, val, test, path_to_save):
    # начальные параметры обучения
    learning_params_df = pd.DataFrame({'random_seed': [seed], 
                       'learning_rate': [lrate], 
                       'batch_size': [batch_size], 
                       'time_to_learn (seconds)': [learn_time]})
    
    # параметры процесса обучения
    learn_df = pd.DataFrame.from_dict(history)
    learn_df.index.name = 'epoch'
    learn_df.columns.name = 'param'
    
    # параметры коллбэков
    callback_df = pd.DataFrame.from_dict(callback_params)
    callback_df.index.name = 'param'
    callback_df.columns.name = 'callback_name'
    
    
    # параметры для подготовки и чтения датасетов
    descs = {key:[{**desc.model_dump(), 'desc_number':i} for i, desc in enumerate(item)] for key, item in dataset_desc.items()}
    dataset_df_list = []
    for key, item in descs.items():
        for item_i in item:
            temp = pd.DataFrame({key:pd.Series(item_i)}).T
            temp.index.name = 'dataset_part'
            dataset_df_list.append(temp)
        
    dataset_df = pd.concat(dataset_df_list, axis=0)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index(['dataset_part', 'run_name', 'desc_number'])
    
    # параметры подготовленных датасетов
    prepared_dataset_df = pd.DataFrame({'Shape':pd.Series({'train': train.shape, 'val': val.shape, 'test': test.shape})})
    prepared_dataset_df.index.name = 'dataset_part'


    with pd.ExcelWriter(path_to_save) as writer:
        learning_params_df.to_excel(writer, sheet_name = 'learning_start_params')
        learn_df.to_excel(writer, sheet_name = 'learning_progress')
        callback_df.to_excel(writer, sheet_name = 'callback_params')
        dataset_df.to_excel(writer, sheet_name = 'dataset_params')
        prepared_dataset_df.to_excel(writer, sheet_name = 'prepared_dataset_shapes')

In [ ]:
cont = False
for seed in range(0, 200, 10):
    print(f'Seed: {seed}',"|"*10)
    tf.compat.v1.set_random_seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    for lrate in [0.01,0.005,0.0025]:
        print(f'Start learning rate: {lrate}',"|"*5)
        for batch_size in [8,16,32,64]:
            print(f'\tBatch size: {batch_size}')

            # get model
            model = get_AE(lrate)

            start = time.time()

            history = model.fit(train, train,
                            batch_size = batch_size, 
                            epochs = 80, 
                            verbose = 0, 
                            shuffle = True,
                            validation_data = (val, val), 
                            callbacks = callback_list)
            
            cur_test_loss = model.evaluate(test, test, batch_size=batch_size, verbose=0)[0]

            learn_time = time.time() - start
            

            cur_train_loss = history.history['loss'][-1]
            cur_val_loss = history.history['val_loss'][-1]
            
            model_name = (
                f"id=v{MODEL_VERSION:04}n{MODEL_NUMBER:04}" +
                f"_in({DECODED_SIZE})_hid({ENCODED_SIZE})" + 
                f"_loss_MSE=(train={cur_train_loss:.5f}," + 
                f"val={cur_val_loss:.5f},test={cur_test_loss:.5f})" + 
                f"_seed={seed}_lrate={lrate}_bach_size={batch_size}_tf={tf.__version__}")


            if cur_train_loss < MIN_TRAIN_LOSS and cur_val_loss < MIN_VAL_LOSS:
                MIN_TRAIN_LOSS = cur_train_loss
                MIN_VAL_LOSS = cur_val_loss
                cont = True
            if cur_val_loss < MIN_VAL_LOSS:
                MIN_VAL_LOSS = cur_val_loss
                cont = True
            if cur_train_loss < MIN_TRAIN_LOSS:
                MIN_TRAIN_LOSS = cur_train_loss
                cont = True

                
            print(f'\t\tEpochs: {len(history.history["loss"])}')
            print(f"\t\tloss_MSE=(train={cur_train_loss:.5f},val={cur_val_loss:.5f},test={cur_test_loss:.5f})")
            print(f'\t\tВремя на обучение модели: {learn_time}')
            
            if cont:
               
                save_learning_params(seed, lrate, batch_size, learn_time, history.history, callback_params, dataset_desc, train, val, test, PATH_TO_SAVE_MODEL_PROGRESS/f"{model_name}_learning_data.xlsx")
                    
                model.save(PATH_TO_SAVE_MODEL/f'{model_name}.keras')
                
                MODEL_NUMBER+=1
                cont=False
    print()